In [1]:
from typing import Final
from pathlib import Path

# Re-scraped 2024 metadata
METADATA_PATH: Final[Path] = Path("../data/raw/2024/train_metadata.csv")
XC_METADATA_PATH: Final[Path] = Path("./dataset/train_metadata_xc.csv")
OUT_METADATA_PATH: Final[Path] = Path("../data/raw/2024/train_metadata_with_datetime.csv")
EXTENSION: Final[str] = ".ogg"

In [6]:
import pandas as pd

metadata = pd.read_csv(METADATA_PATH)
xc_metadata = pd.read_csv(XC_METADATA_PATH)

metadata["id"] = metadata["filename"].str.replace(EXTENSION, "").str.split("/").str[-1].str[2:].astype(int)
metadata = metadata.merge(xc_metadata[["id", "length", "time", "date", "uploaded"]], on="id")
metadata.head()

/tmp/ipykernel_3624411/2822668448.py:4: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  xc_metadata = pd.read_csv(XC_METADATA_PATH)


,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename,id,length,time,date,uploaded
0,asbfly,[],['call'],39.2297,118.1987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,asbfly/XC134896.ogg,134896,0:27,17:30,2013-05-03,2013-05-26
1,asbfly,[],['song'],36.3319,127.3555,Muscicapa dauurica,Asian Brown Flycatcher,Stuart Fisher,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/175797,asbfly/XC175797.ogg,175797,0:29,08:00,2014-04-22,2014-04-23
2,asbfly,[],['call'],21.1697,70.6005,Muscicapa dauurica,Asian Brown Flycatcher,vir joshi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/207738,asbfly/XC207738.ogg,207738,0:14,12:00,2014-11-22,2014-12-24
3,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209218,asbfly/XC209218.ogg,209218,0:47,12:30,2014-11-20,2015-01-03
4,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209219,asbfly/XC209219.ogg,209219,0:30,12:30,2014-11-20,2015-01-03


In [28]:
import re

times = list(metadata["time"].astype(str).unique())

def parse_time(time_str: str) -> str | None:
    time_str = str(time_str).strip().lower()
    
    # Handling special cases
    if time_str in ["nan", "null", "?", "?:?", "??:??", "x:x", "xx:xx"]:
        return None
    elif time_str == "am":
        return "06:00:00"
    elif time_str == "sunset":
        return "19:00:00"  # Assuming sunset time as 19:00
    elif time_str == "dawn":
        return "06:00:00"  # Assuming dawn time as 06:00
    
    # Handling AM/PM times
    am_pm_match = re.match(r"(\d{1,2}):(\d{2})\s?(am|pm)", time_str)
    if am_pm_match:
        hour, minute, period = am_pm_match.groups()
        hour = int(hour)
        minute = int(minute)
        if period == "pm" and hour != 12:
            hour += 12
        elif period == "am" and hour == 12:
            hour = 0
        return f"{hour:02}:{minute:02}:00"
    
    # Handling HH:MM:SS and HH:MM times
    hh_mm_ss_match = re.match(r"(\d{1,2}):(\d{2})(?::(\d{2}))?", time_str)
    if hh_mm_ss_match:
        hour, minute, second = hh_mm_ss_match.groups()
        hour = int(hour)
        minute = int(minute)
        second = int(second) if second else 0
        return f"{hour:02}:{minute:02}:{second:02}"
    
    raise ValueError(f"Unknown time format: {time_str}")

metadata["normalized_time"] = metadata["time"].astype(str).apply(parse_time)
metadata

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename,id,length,time,date,uploaded,normalized_time
0,asbfly,[],['call'],39.2297,118.1987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,asbfly/XC134896.ogg,134896,0:27,17:30,2013-05-03,2013-05-26,17:30:00
1,asbfly,[],['song'],36.3319,127.3555,Muscicapa dauurica,Asian Brown Flycatcher,Stuart Fisher,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/175797,asbfly/XC175797.ogg,175797,0:29,08:00,2014-04-22,2014-04-23,08:00:00
2,asbfly,[],['call'],21.1697,70.6005,Muscicapa dauurica,Asian Brown Flycatcher,vir joshi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/207738,asbfly/XC207738.ogg,207738,0:14,12:00,2014-11-22,2014-12-24,12:00:00
3,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209218,asbfly/XC209218.ogg,209218,0:47,12:30,2014-11-20,2015-01-03,12:30:00
4,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209219,asbfly/XC209219.ogg,209219,0:30,12:30,2014-11-20,2015-01-03,12:30:00


In [52]:
from datetime import datetime


def combine_date_time(row):
    date_str = row['date']
    time_str = row['normalized_time']
    
    if (not date_str or pd.isna(date_str) or date_str == "0000-00-00") and (not time_str):
        return None
    
    if not date_str or pd.isna(date_str):
        date_str = '0001-01-01'
        
    if date_str[0:4] == "0000":
        date_str = "0001" + date_str[4:]
        
    if date_str[5:7] == "00":
        date_str = date_str[:5] + "01" + date_str[7:]
        
    if date_str.endswith("00"):
        date_str = date_str[:-2] + "01"
    
    if not time_str or pd.isna(time_str):
        time_str = '00:00:00'
        
    try:
        combined_datetime = datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M:%S")
    except ValueError:
        print(f"Error: {date_str} {time_str}")
        combined_datetime = None
    
    return combined_datetime

metadata["datetime"] = metadata.apply(combine_date_time, axis=1)
metadata

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename,id,length,time,date,uploaded,normalized_time,datetime
0,asbfly,[],['call'],39.2297,118.1987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,asbfly/XC134896.ogg,134896,0:27,17:30,2013-05-03,2013-05-26,17:30:00,2013-05-03 17:30:00
1,asbfly,[],['song'],36.3319,127.3555,Muscicapa dauurica,Asian Brown Flycatcher,Stuart Fisher,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/175797,asbfly/XC175797.ogg,175797,0:29,08:00,2014-04-22,2014-04-23,08:00:00,2014-04-22 08:00:00
2,asbfly,[],['call'],21.1697,70.6005,Muscicapa dauurica,Asian Brown Flycatcher,vir joshi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/207738,asbfly/XC207738.ogg,207738,0:14,12:00,2014-11-22,2014-12-24,12:00:00,2014-11-22 12:00:00
3,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209218,asbfly/XC209218.ogg,209218,0:47,12:30,2014-11-20,2015-01-03,12:30:00,2014-11-20 12:30:00
4,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209219,asbfly/XC209219.ogg,209219,0:30,12:30,2014-11-20,2015-01-03,12:30:00,2014-11-20 12:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24209,zitcis1,[],[''],43.5925,4.5434,Cisticola juncidis,Zitting Cisticola,Chèvremont Fabian,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://xeno-canto.org/845747,zitcis1/XC845747.ogg,845747,1:02,07:00,2023-07-11,2023-12-06,07:00:00,2023-07-11 07:00:00
24210,zitcis1,[],[''],43.5925,4.5434,Cisticola juncidis,Zitting Cisticola,Chèvremont Fabian,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/845817,zitcis1/XC845817.ogg,845817,0:10,08:30,2023-07-09,2023-12-06,08:30:00,2023-07-09 08:30:00
24211,zitcis1,[],[''],51.1207,4.5607,Cisticola juncidis,Zitting Cisticola,Wim Jacobs,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/856176,zitcis1/XC856176.ogg,856176,0:10,?,2021-09-19,2023-12-17,None,2021-09-19 00:00:00
24212,zitcis1,[],[''],41.5607,-8.4236,Cisticola juncidis,Zitting Cisticola,Jorge Leitão,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://xeno-canto.org/856723,zitcis1/XC856723.ogg,856723,0:24,11:30,2023-12-18,2023-12-19,11:30:00,2023-12-18 11:30:00


datetime.datetime(1, 1, 1, 0, 0)